# Python入門

この実験を受ける学生はすでに一度Pythonに触れているということを前提しています．

本授業はプログラミングの授業ではないため，Pythonの丁寧な導入はできませんが，ここではいくつか大事な点を説明したいと思います．

なおPythonの基礎の自習には，次の素晴らしい資料あります：
https://utokyo-ipp.github.io/index.html

### Pythonの特徴：インデントが大事
次のプログラムは，0から99までを足し上げるプログラムです．`for`の始まりから終わりは，インデント（`\tab`によるスペース）で表現されてます．

In [1]:
s = 0
for i in range(100):  # 0, 1, ..., 99の和
    s = s + i   # for の内部
print(s)  # forの外部

4950


次のプログラムは上と似ていますが，最後の`print`が`for`の内部に入っています．実行してみると上との違いがわかるでしょう．

In [ ]:
s = 0
for i in range(100):  # 0, 1, ..., 99の和
    s = s + i   # forの内部
    print(s)    # forの内部

### Pythonにおける関数定義
何か入力を受けつけてそれを処理し，出力するものを関数と呼びます（入力や出力がない場合もあります）．

次の関数は，入力として`x`と`y`を受けて，$x/y$を返す関数です．ただし，$y=0$の場合は0を返します．

In [3]:
def protected_division(x, y):
    if y == 0:        # if-else分の書き方もついでに覚えましょう
        return 0     
    else: 
        return x / y 

In [4]:
x, y = 10, 5
z = protected_division(x, y)
print(f'{x} / {y} = {z}')

10 / 5 = 2.0


### Pythonにおけるクラスの定義

「クラス」というのはまだ触れていないかもしれませんが，プログラミングにおいて非常に重要な概念です．

本実験ではPytorchというライブラリを用いますが，そこで用いる機能のほとんど全ては「クラス」であると言っていいでしょう．

クラスというのは，大雑把には「変数や関数をひとまとめにしたもの」です．ひとまとめにした方が良さそうな変数や機能の束です．

例えば次では，学生ごとの英語と数学の点数を管理するために`Student`クラスを考えています．

In [5]:
class Student(): 
    # __init__は初期化の関数
    def __init__(self, name, english=0, math=0):  # 英語と数学の点数が0に初期設定されている．
        self.name = name
        self.english = english
        self.math = math
    
    def best_score(self): 
        return max(self.english, self.math)
    
    def worst_score(self): 
        return min(self.english, self.math)

    def average_score(self):
        return (self.english + self.math)/2 

学生の名前と各教科の点数はひとまとめにした方が自然です．また評価のために，最高・最低・平均が必要になるかもしれないので，それらを計算する関数（メソッド）が実装されています．どの関数にも`self`があるのは約束だと思ってください．次のようにして，別々の学生（「学生オブジェクト」）を作ることができます．

In [6]:
s1 = Student('Saikawa', english=50, math=80)
s2 = Student('Magata', english=80, math=100) 

In [7]:
print(s1.name)
print(s1.english)
print(s1.average_score())

Saikawa
50
65.0


本実験ではPytorchライブラリを使いますが，特に`Net`クラスはよく用います．次を見てみましょう．

In [7]:
import torch                       # Pytorchを導入
import torch.nn as nn              # Pytorchのサブモジュールのnnを導入

# クラスの定義
class Net(nn.Module):       # nn.Moduleクラスを「継承」（雛形として採用している）
    def __init__(self):     # 初期化の関数（オブジェクト生成時に自動的に実行される）  
        super().__init__()  # nn.Moduleクラスの初期化
        self.fc = nn.Linear(2, 5)  # 2次元の行ベクトルを5次元の行ベクトルに変換する関数
        '''
        nn.Linear(m, n)は，m次元行ベクトルxを受け取ってn次元行ベクトルを返す変換（つまり以下の変換）を用意する関数
            z = xA + b 
        ここで，A: m x n行列，b:n次元行ベクトル
        '''

    # ネットワークにデータを通す時に機能する
    def forward(self, x):
        x = self.fc(x)
        return x

# Netオブジェクトの生成．__init__が実行される．
model = Net()
print('ネットワークの構造')
print(model, end='\n\n')
print('ネットワークの重みパラメータ')
print(model.fc.weight, end='\n\n')

# Tensorオブジェクトの生成
x = torch.tensor([1.0, 2.0])  # 2次元行ベクトル

# xを5次元に変換
z = model.forward(x)  # データxを代入．
z = model(x)          # これも同じ意味（nn.Moduleクラスの仕様）

print(f'入力：{x}')
print(f'出力：{z}')


ネットワークの構造
Net(
  (fc): Linear(in_features=2, out_features=5, bias=True)
)

ネットワークの重みパラメータ
Parameter containing:
tensor([[ 0.0868, -0.1232],
        [-0.4293, -0.1200],
        [-0.2073,  0.4531],
        [ 0.3399, -0.5321],
        [ 0.5523,  0.3543]], requires_grad=True)

入力：tensor([1., 2.])
出力：tensor([ 0.1995, -0.5437,  0.7452, -1.0397,  0.6762], grad_fn=<AddBackward0>)


上に関しては込み入っているの実験の際に説明しますが，次のような点に着目してください．

- `Net`クラスは`nn.Module`クラスを雛形にしています．`__init__`や`forward`は`Net`のために新たに定義していますが，その他は（裏で）`nn.Module`によって定義されています．
- `nn.Linear`も別のクラス（これも`nn.Module`を雛形にしている）
- `torch.tensor`もクラスで，`x`はtensorオブジェクト

特に`tensor`オブジェクトに関してもう少し述べます．

テンソル（Tensor）とは機械学習では多次元配列のことを指します．0次テンソルはスカラ，1次テンソルはベクトル，2次テンソルは行列...といった具合です．

In [39]:
a = torch.tensor(1)  # 0次のテンソル
A = torch.tensor([ [1, 2],
                   [4, 3] ])  # 2次のテンソル

A.T @ A    # Aの転置行列とAの積（行列やベクトルの積は@）

tensor([[17, 14],
        [14, 13]])

`tensor`クラスにはさまざまなメソッドがあります．

In [43]:
print(A)
print(A.shape)      # ---> torch.Size([2, 2])
print(A.flatten())  # ---> tensor([1, 2, 3, 4])
print(A.max())      # Aの要素の最大値
print(A.argmax())   # Aの要素の最大値の位置
print(A.argmax(dim=0)) # Aの各列の最大値の位置
print(A.argmax(dim=1)) # Aの各行の最大値の位置

tensor([[1, 2],
        [4, 3]])
torch.Size([2, 2])
tensor([1, 2, 4, 3])
tensor(4)
tensor(2)
tensor([1, 1])
tensor([1, 0])


今自分が扱っているクラスにどのようなメソッドが定義されているか，必要に応じて確認しましょう．

例：「pytorch tensor document」で検索すると，公式のドキュメント[https://pytorch.org/docs/stable/tensors.html]が見つかる．